In [1]:
import warnings
warnings.filterwarnings('ignore')

#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential


In [4]:
from tensorflow.keras.layers import LSTM, Dense, Embedding

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
text = """
Once upon a time in a land far away, there was a little village surrounded by lush green forests.
The villagers lived in harmony with nature, cherishing the beauty around them.
"""

In [9]:
tokenizer = Tokenizer(char_level=True)

In [10]:
tokenizer.fit_on_texts(text)

In [11]:
char_index = tokenizer.word_index

In [12]:
index_char = tokenizer.index_word

In [13]:
vocab_size = len(char_index) + 1

In [14]:
sequences = tokenizer.texts_to_sequences(text)

In [15]:
sequences = [item for sublist in sequences for item in sublist]

In [16]:
seq_length = 40

In [17]:
X,y = [],[]

In [18]:
for i in range(len(sequences) - seq_length):
    X.append(sequences[i:i + seq_length])
    y.append(sequences[i + seq_length])

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)

In [19]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length),
    LSTM(150, return_sequences=False),
    Dense(vocab_size, activation='softmax')#Activation tanh
])

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Step 3: Training the model
model.fit(X, y, epochs=100, batch_size=64)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - accuracy: 0.0895 - loss: 3.2538
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.1506 - loss: 3.2294
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.1408 - loss: 3.1878
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.1350 - loss: 3.0466
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.1428 - loss: 3.1070
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.1486 - loss: 3.0087 
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.1486 - loss: 2.9298
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.1564 - loss: 2.9135
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 0.1604 - loss: 2.9294
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.1447 - loss: 2.9126
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.1408 - loss: 2.9170
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.148

In [21]:
def generate_text(seed_text, num_chars):
    for _ in range(num_chars):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, truncating='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_char_index = np.argmax(predicted_probs)
        predicted_char = index_char[predicted_char_index]
        seed_text += predicted_char
    return seed_text

In [22]:
seed_text = "Once upon a time"
generated_text = generate_text(seed_text, num_chars=200)
print("Generated text:\n", generated_text)

Generated text:
 Once upon a time vilitle lilage surrounded by lus lgeren foresstt

the vilaage surroundd by lus lgeren foresstthe 
he vilage surrondd bish luge nreren fostt
the vililage surroundd by lus lgeren foresstt

the vilaage 


In [24]:
import gradio as gr

In [25]:
def gradio_generate(seed_text, num_chars):
    return generate_text(seed_text, int(num_chars))

interface = gr.Interface(
    fn=gradio_generate,
    inputs=[
        gr.Textbox(label="Seed Text", placeholder="Enter seed text, e.g., 'Once upon a time'"),
        gr.Slider(minimum=50, maximum=500, step=10, value=200, label="Number of Characters")
    ],
    outputs=gr.Textbox(label="Generated Text"),
    title="LSTM-Based Text Generator",
    description="Enter a seed text and specify the number of characters to generate new text using a LSTM-based neural network."
)

# Launch the Gradio app
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://613d2834007c2c5e5f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
